## CatBoost-Optunaのサンプルコード（train, valid, test使用）

In [1]:
%load_ext lab_black

In [2]:
# ライブラリーのインポート
import os

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# ボストンの住宅価格データ
from sklearn.datasets import load_boston

# 前処理
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# CatBoost
import catboost as cb
from catboost import CatBoost, Pool

# Optuna
import optuna
from optuna.samplers import TPESampler

# 評価指標
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=pd.core.common.SettingWithCopyWarning)

In [3]:
print(cb.__version__)

0.26


In [4]:
print(optuna.__version__)

2.8.0


In [5]:
# データセットの読込み
boston = load_boston()

# 説明変数の格納
df = pd.DataFrame(boston.data, columns=boston.feature_names)
# 目的変数の追加
df["MEDV"] = boston.target

# データの中身を確認
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


#### 前処理

In [6]:
# ランダムシード値
RANDOM_STATE = 10

# 学習データと評価データの割合
TEST_SIZE = 0.2

# 学習データと評価データを作成
x_train, x_test, y_train, y_test = train_test_split(
    df.iloc[:, 0 : df.shape[1] - 1],
    df.iloc[:, df.shape[1] - 1],
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
)

# trainのデータセットの2割をモデル学習時のバリデーションデータとして利用する
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

In [7]:
def objective(trial):

    param = {
        "iterations": trial.suggest_int("iterations", 50, 300),
        "depth": trial.suggest_int("depth", 4, 10),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),
        "random_strength": trial.suggest_int("random_strength", 0, 100),
        "bagging_temperature": trial.suggest_loguniform(
            "bagging_temperature", 0.01, 100.00
        ),
        "od_type": trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        "od_wait": trial.suggest_int("od_wait", 10, 50),
    }

    model = cb.CatBoostRegressor(**param)

    model.fit(
        x_train,
        y_train,
        eval_set=[(x_valid, y_valid)],
        early_stopping_rounds=100,
        verbose=False,
    )

    preds = model.predict(x_valid)
    mae = mean_absolute_error(y_valid, preds)

    return mae

In [8]:
%%time
# optunaで最適値を見つける
# create_studyメソッドの引数"sampler"にサンプラーと乱数シードを指定
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=RANDOM_STATE))
study.optimize(objective, n_trials=50)

[I 2021-07-08 06:37:03,686] A new study created in memory with name: no-name-fcd3cd6f-12f9-49e9-98cf-3ccf3bb16875
[I 2021-07-08 06:37:05,163] Trial 0 finished with value: 1.9714684778872467 and parameters: {'iterations': 243, 'depth': 4, 'learning_rate': 0.08629294202140579, 'random_strength': 75, 'bagging_temperature': 0.986343187233007, 'od_type': 'IncToDec', 'od_wait': 41}. Best is trial 0 with value: 1.9714684778872467.
[I 2021-07-08 06:37:05,736] Trial 1 finished with value: 2.259837967684255 and parameters: {'iterations': 92, 'depth': 4, 'learning_rate': 0.1028867751008824, 'random_strength': 96, 'bagging_temperature': 0.01037034167132714, 'od_type': 'Iter', 'od_wait': 35}. Best is trial 0 with value: 1.9714684778872467.
[I 2021-07-08 06:37:07,359] Trial 2 finished with value: 2.049156827287779 and parameters: {'iterations': 231, 'depth': 6, 'learning_rate': 0.22681076499740013, 'random_strength': 72, 'bagging_temperature': 1.4797129411694714, 'od_type': 'Iter', 'od_wait': 37}. B

CPU times: user 38.4 s, sys: 13.1 s, total: 51.5 s
Wall time: 1min 2s


In [9]:
best_params = study.best_params
best_params["random_state"] = RANDOM_STATE
best_params

{'iterations': 250,
 'depth': 6,
 'learning_rate': 0.09108179261805606,
 'random_strength': 99,
 'bagging_temperature': 0.6169776013615714,
 'od_type': 'IncToDec',
 'od_wait': 22,
 'random_state': 10}

In [10]:
# チューニングしたハイパーパラメーターをフィット
optimised_model = cb.CatBoostRegressor(**(best_params))

optimised_model.fit(x_train, y_train)

# CatBoost推論
y_pred = optimised_model.predict(x_test)

0:	learn: 8.9204450	total: 2.02ms	remaining: 504ms
1:	learn: 8.7041363	total: 3.3ms	remaining: 409ms
2:	learn: 8.5062649	total: 5.19ms	remaining: 428ms
3:	learn: 8.3402085	total: 6.04ms	remaining: 371ms
4:	learn: 8.1589345	total: 7.52ms	remaining: 368ms
5:	learn: 8.0326157	total: 8.7ms	remaining: 354ms
6:	learn: 7.8789683	total: 10.1ms	remaining: 349ms
7:	learn: 7.7506962	total: 11ms	remaining: 334ms
8:	learn: 7.6177758	total: 13.7ms	remaining: 366ms
9:	learn: 7.4866367	total: 17.8ms	remaining: 427ms
10:	learn: 7.3134388	total: 19.3ms	remaining: 420ms
11:	learn: 7.1004197	total: 20.4ms	remaining: 404ms
12:	learn: 7.0256128	total: 21.3ms	remaining: 388ms
13:	learn: 6.9913812	total: 21.8ms	remaining: 367ms
14:	learn: 6.8933628	total: 23ms	remaining: 360ms
15:	learn: 6.6947685	total: 24ms	remaining: 351ms
16:	learn: 6.5410892	total: 25.1ms	remaining: 344ms
17:	learn: 6.4317160	total: 26ms	remaining: 335ms
18:	learn: 6.3808467	total: 26.8ms	remaining: 326ms
19:	learn: 6.2220680	total: 27.7

In [11]:
# 評価
def calculate_scores(true, pred):
    """全ての評価指標を計算する

    Parameters
    ----------
    true (np.array)       : 実測値
    pred (np.array)       : 予測値

    Returns
    -------
    scores (pd.DataFrame) : 各評価指標を纏めた結果

    """
    scores = {}
    scores = pd.DataFrame(
        {
            "R2": r2_score(true, pred),
            "MAE": mean_absolute_error(true, pred),
            "MSE": mean_squared_error(true, pred),
            "RMSE": np.sqrt(mean_squared_error(true, pred)),
        },
        index=["scores"],
    )
    return scores

In [12]:
scores = calculate_scores(y_test, y_pred)
print(scores)

              R2       MAE        MSE     RMSE
scores  0.863433  2.596167  14.282276  3.77919
